# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd

import nltk
nltk.download('punkt')
nltk.download('wordnet')

from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
x_columns = ['id','message','original','genre']
engine = create_engine('sqlite:///messages_clean.db')
df = pd.read_sql_table('SampleTable',engine)
X = df['message']
Y = df.drop(x_columns, axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for token in tokens:
        clean_tok = lemmatizer.lemmatize(token).lower().strip()
        clean_tokens.append(clean_tok)
        
    return(clean_tokens)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('multi', MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
model = pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)

for i,col in enumerate(y_test.columns):
    y_pred_i = y_pred.T[i]
    y_test_i = y_test[col]
    print('*** {}. Classification report for {}'.format(i+1,col))
    print(classification_report(y_test_i, y_pred_i))

*** 1. Classification report for related
             precision    recall  f1-score   support

          0       0.61      0.35      0.45      1206
          1       0.82      0.93      0.87      4000
          2       0.33      0.08      0.13        38

avg / total       0.77      0.79      0.77      5244

*** 2. Classification report for request
             precision    recall  f1-score   support

          0       0.88      0.99      0.93      4329
          1       0.87      0.36      0.51       915

avg / total       0.88      0.88      0.86      5244

*** 3. Classification report for offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5216
          1       0.00      0.00      0.00        28

avg / total       0.99      0.99      0.99      5244

*** 4. Classification report for aid_related
             precision    recall  f1-score   support

          0       0.72      0.89      0.79      3019
          1       0.77      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7ff15003c268>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('multi',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=Non

```python
Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fb45020e9e0>)),
                ('tfidf', TfidfTransformer(norm='l1', smooth_idf=False)),
                ('multi',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))],
         verbose=True)
```

In [9]:
parameters = {'multi__estimator__bootstrap' : [True, False],
              'multi__estimator__max_depth' : [5, 7, 9],
              'multi__estimator__min_samples_leaf' : [3, 5, 7],
              'multi__estimator__n_estimators' : [50, 100, 200, 300]}

cv = GridSearchCV(pipeline,param_grid=parameters, verbose=15)
cv.fit(X_train, y_train)


Fitting 3 folds for each of 72 candidates, totalling 216 fits
[CV] multi__estimator__bootstrap=True, multi__estimator__max_depth=5, multi__estimator__min_samples_leaf=3, multi__estimator__n_estimators=50 
[CV]  multi__estimator__bootstrap=True, multi__estimator__max_depth=5, multi__estimator__min_samples_leaf=3, multi__estimator__n_estimators=50, score=0.1988270633671864, total=  20.9s
[CV] multi__estimator__bootstrap=True, multi__estimator__max_depth=5, multi__estimator__min_samples_leaf=3, multi__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   32.0s remaining:    0.0s


[CV]  multi__estimator__bootstrap=True, multi__estimator__max_depth=5, multi__estimator__min_samples_leaf=3, multi__estimator__n_estimators=50, score=0.19997139178944356, total=  20.8s
[CV] multi__estimator__bootstrap=True, multi__estimator__max_depth=5, multi__estimator__min_samples_leaf=3, multi__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.1min remaining:    0.0s


[CV]  multi__estimator__bootstrap=True, multi__estimator__max_depth=5, multi__estimator__min_samples_leaf=3, multi__estimator__n_estimators=50, score=0.19399141630901287, total=  21.1s
[CV] multi__estimator__bootstrap=True, multi__estimator__max_depth=5, multi__estimator__min_samples_leaf=3, multi__estimator__n_estimators=100 


...

[CV]  multi__estimator__bootstrap=True, multi__estimator__max_depth=5, multi__estimator__min_samples_leaf=5, multi__estimator__n_estimators=50, score=0.19968530968387926, total=  21.0s
[CV] multi__estimator__bootstrap=True, multi__estimator__max_depth=5, multi__estimator__min_samples_leaf=5, multi__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Done  14 out of  14 | elapsed: 13.6min remaining:    0.0s


[CV]  multi__estimator__bootstrap=True, multi__estimator__max_depth=5, multi__estimator__min_samples_leaf=5, multi__estimator__n_estimators=50, score=0.19399141630901287, total=  21.0s
[CV] multi__estimator__bootstrap=True, multi__estimator__max_depth=5, multi__estimator__min_samples_leaf=5, multi__estimator__n_estimators=100 
[CV]  multi__estimator__bootstrap=True, multi__estimator__max_depth=5, multi__estimator__min_samples_leaf=5, multi__estimator__n_estimators=100, score=0.1991131454727507, total=  30.9s
[CV] multi__estimator__bootstrap=True, multi__estimator__max_depth=5, multi__estimator__min_samples_leaf=5, multi__estimator__n_estimators=100 
[CV]  multi__estimator__bootstrap=True, multi__estimator__max_depth=5, multi__estimator__min_samples_leaf=5, multi__estimator__n_estimators=100, score=0.19997139178944356, total=  30.2s
[CV] multi__estimator__bootstrap=True, multi__estimator__max_depth=5, multi__estimator__min_samples_leaf=5, multi__estimator__n_estimators=100 
[CV]  multi_

[Parallel(n_jobs=1)]: Done 216 out of 216 | elapsed: 253.3min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'multi__estimator__bootstrap': [True, False], 'multi__estimator__max_depth': [5, 7, 9], 'multi__estimator__min_samples_leaf': [3, 5, 7], 'multi__estimator__n_estimators': [50, 100, 200, 300]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=15)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
y_pred = cv.best_estimator_.predict(X_test)

for i,col in enumerate(y_test.columns):
    y_pred_i = y_pred.T[i]
    y_test_i = y_test[col]
    print('*** {}. Classification report for {}'.format(i+1,col))
    print(classification_report(y_test_i, y_pred_i))

*** 1. Classification report for related
             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1206
          1       0.76      1.00      0.87      4000
          2       0.00      0.00      0.00        38

avg / total       0.58      0.76      0.66      5244

*** 2. Classification report for request
             precision    recall  f1-score   support

          0       0.83      1.00      0.90      4329
          1       0.00      0.00      0.00       915

avg / total       0.68      0.83      0.75      5244

*** 3. Classification report for offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5216
          1       0.00      0.00      0.00        28

avg / total       0.99      0.99      0.99      5244

*** 4. Classification report for aid_related
             precision    recall  f1-score   support

          0       0.58      1.00      0.73      3019
          1       1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [5]:
alternate_pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('multi', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
model2 = alternate_pipeline.fit(X_train, y_train)

In [7]:
y_pred = alternate_pipeline.predict(X_test)

for i,col in enumerate(y_test.columns):
    y_pred_i = y_pred.T[i]
    y_test_i = y_test[col]
    print('*** {}. Classification report for {}'.format(i+1,col))
    print(classification_report(y_test_i, y_pred_i))

*** 1. Classification report for related
             precision    recall  f1-score   support

          0       0.60      0.31      0.41      1206
          1       0.81      0.94      0.87      4002
          2       0.00      0.00      0.00        36

avg / total       0.76      0.79      0.76      5244

*** 2. Classification report for request
             precision    recall  f1-score   support

          0       0.91      0.96      0.93      4317
          1       0.76      0.54      0.63       927

avg / total       0.88      0.89      0.88      5244

*** 3. Classification report for offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5220
          1       0.00      0.00      0.00        24

avg / total       0.99      0.99      0.99      5244

*** 4. Classification report for aid_related
             precision    recall  f1-score   support

          0       0.74      0.87      0.80      3003
          1       0.77      0

In [8]:
alternate_pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f4c20174620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('multi',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_ra

In [9]:
parameters = {'multi__estimator__learning_rate' : [0.9, 1],
              'multi__estimator__n_estimators' : [100, 200, 300]}

cv2 = GridSearchCV(alternate_pipeline,param_grid=parameters, verbose=15)
cv2.fit(X_train, y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] multi__estimator__learning_rate=0.9, multi__estimator__n_estimators=100 
[CV]  multi__estimator__learning_rate=0.9, multi__estimator__n_estimators=100, score=0.23387212129881277, total= 4.0min
[CV] multi__estimator__learning_rate=0.9, multi__estimator__n_estimators=100 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.3min remaining:    0.0s


[CV]  multi__estimator__learning_rate=0.9, multi__estimator__n_estimators=100, score=0.23229866971820912, total= 4.0min
[CV] multi__estimator__learning_rate=0.9, multi__estimator__n_estimators=100 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  8.7min remaining:    0.0s


[CV]  multi__estimator__learning_rate=0.9, multi__estimator__n_estimators=100, score=0.24763948497854077, total= 4.1min
[CV] multi__estimator__learning_rate=0.9, multi__estimator__n_estimators=200 


...

[CV]  multi__estimator__learning_rate=1, multi__estimator__n_estimators=200, score=0.22357316549849807, total= 8.0min
[CV] multi__estimator__learning_rate=1, multi__estimator__n_estimators=200 


[Parallel(n_jobs=1)]: Done  14 out of  14 | elapsed: 105.9min remaining:    0.0s


[CV]  multi__estimator__learning_rate=1, multi__estimator__n_estimators=200, score=0.23047210300429186, total= 8.0min
[CV] multi__estimator__learning_rate=1, multi__estimator__n_estimators=300 
[CV]  multi__estimator__learning_rate=1, multi__estimator__n_estimators=300, score=0.19782577599771134, total=11.8min
[CV] multi__estimator__learning_rate=1, multi__estimator__n_estimators=300 
[CV]  multi__estimator__learning_rate=1, multi__estimator__n_estimators=300, score=0.20912601916750106, total=11.9min
[CV] multi__estimator__learning_rate=1, multi__estimator__n_estimators=300 
[CV]  multi__estimator__learning_rate=1, multi__estimator__n_estimators=300, score=0.22203147353361946, total=12.0min


[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed: 152.1min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'multi__estimator__learning_rate': [0.9, 1], 'multi__estimator__n_estimators': [100, 200, 300]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=15)

In [10]:
y_pred = cv2.best_estimator_.predict(X_test)

for i,col in enumerate(y_test.columns):
    y_pred_i = y_pred.T[i]
    y_test_i = y_test[col]
    print('*** {}. Classification report for {}'.format(i+1,col))
    print(classification_report(y_test_i, y_pred_i))

*** 1. Classification report for related
             precision    recall  f1-score   support

          0       0.51      0.38      0.44      1206
          1       0.82      0.89      0.86      4002
          2       0.50      0.19      0.28        36

avg / total       0.75      0.77      0.76      5244

*** 2. Classification report for request
             precision    recall  f1-score   support

          0       0.91      0.96      0.94      4317
          1       0.76      0.58      0.66       927

avg / total       0.89      0.89      0.89      5244

*** 3. Classification report for offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5220
          1       0.00      0.00      0.00        24

avg / total       0.99      0.99      0.99      5244

*** 4. Classification report for aid_related
             precision    recall  f1-score   support

          0       0.77      0.85      0.81      3003
          1       0.77      0

In [11]:
cv2.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ator=None,
          learning_rate=0.9, n_estimators=200, random_state=None),
           n_jobs=1))])

### 9. Export your model as a pickle file

In [17]:
import pickle
filename = 'final_model_randomforest.sav'
pickle.dump(cv.best_estimator_, open(filename, 'wb'))

In [19]:
loaded_model1 = pickle.load(open(filename, 'rb'))
result = loaded_model1.score(X_test, y_test)
print(result)

0.694508009153


In [12]:
import pickle
filename = 'final_model_adaboost.sav'
pickle.dump(cv2.best_estimator_, open(filename, 'wb'))

In [13]:
loaded_model2 = pickle.load(open(filename, 'rb'))
result = loaded_model2.score(X_test, y_test)
print(result)

0.238939740656


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.